# Ingest SEC DERA data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [ ]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib

Load Environment Variables

In [ ]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)

In [ ]:
s3_trino = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ["S3_DEV_ENDPOINT"],
    aws_access_key_id=os.environ["S3_DEV_ACCESS_KEY"],
    aws_secret_access_key=os.environ["S3_DEV_SECRET_KEY"],
)
bucket = s3_trino.Bucket(os.environ["S3_DEV_BUCKET"])
bucket.name

Open a Trino connection using JWT for authentication

In [ ]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sec_dera'

In [ ]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine))
connection = engine.connect()

In [ ]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

Enter the Pandas!

In [ ]:
import pandas as pd
import io

In [ ]:
# import osc_ingest_trino as osc

_p2smap = {
    'string': 'varchar',
    'float32': 'real',
    'Float32': 'real',
    'float64': 'double',
    'Float64': 'double',
    'int32': 'integer',
    'Int32': 'integer',
    'int64': 'bigint',
    'Int64': 'bigint',
    'bool': 'boolean',
    'category': 'varchar',
    'datetime64[ns, UTC]': 'timestamp',
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def create_table_schema_pairs(df):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("df must be a pandas DataFrame")
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

Prepare GLEIF matching data

In [ ]:
gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'mtiemann-GLEIF/DERA-matches.csv')
gleif_file.download_file(f'/tmp/dera-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/dera-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = dict(zip(gleif_df.name, gleif_df.LEI))

Drop previous tables and schema to start with a fresh slate

In [ ]:
for ingest_table in [ 'sub', 'num', 'tag', 'ticker' ]:
    sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
    print(sql)
    qres = engine.execute(sql)
    print(qres.fetchall())

sql = f"""
drop schema if exists {ingest_catalog}.{ingest_schema}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema {ingest_catalog}.{ingest_schema}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

Load `ticker` file (updated sporadically from https://www.sec.gov/include/ticker.txt)

In [ ]:
ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'SEC-DERA/ticker.txt')
ticker_file.download_file(f'/tmp/dera-ticker.txt')
ticker_df = pd.read_csv(f'/tmp/dera-ticker.txt', names=['ticker', 'cik'], header=None, sep='\t', dtype={'ticker':'string','cik':'int64'}, engine='c')
ticker_dict = dict(zip(ticker_df.cik, ticker_df.ticker))

In [ ]:
buf = io.BytesIO()
ticker_df.to_parquet(path=buf)
buf.seek(0)
bucket.upload_fileobj(Fileobj=buf,
                      Key=f'trino/{ingest_schema}/ticker/data.parquet')

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.ticker;
create table {ingest_catalog}.{ingest_schema}.ticker(
    name varchar,
    cik bigint
) with (
    format = 'parquet',
    external_location = 's3a://{bucket.name}/trino/{ingest_schema}/ticker/'
);
select count (*) from {ingest_catalog}.{ingest_schema}.ticker
"""
for sql_stmt in sql.split(';'):
    print(sql_stmt)
    qres = engine.execute(sql_stmt)
    print(qres.fetchall())

Load the SUB, NUM, and TAG tables into Trino

In [ ]:
import re
import uuid

# Add a unique identifier to the data set
uid = str(uuid.uuid4())

dera_regex = re.compile(r' ?/.*$')

quarters = ['2020q1', '2020q2', '2020q3', '2020q4', '2021q1', '2021q2', '2021q3']

def ingest_dera_table(qtr, tbl):    
    src_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],f'SEC-DERA/{qtr}/{tbl}.txt')
    timestamp = src_file.last_modified.isoformat()
    src_file.download_file(f'/tmp/dera-{tbl}-{timestamp}.csv')
    df = pd.read_csv(f'/tmp/dera-{tbl}-{timestamp}.csv', header=0, sep='\t', dtype='string', keep_default_na=False, nrows = None, engine='c')
    
    df['uuid'] = uid
    df['quarter'] = qtr
    df = df.convert_dtypes (infer_objects=False, convert_string=True, convert_integer=False, convert_boolean=False, convert_floating=False)
    # Print the output
    # print(df.dtypes)
    
    if tbl=='sub':
        df.name = df.name.map(lambda x: re.sub(dera_regex, '', x))
        df.name = df.name.astype('string')
        df['LEI'] = df.name.map(gleif_dict)
        df.LEI = df.LEI.astype('string')
        df.cik = df.cik.astype('int32')
        df.loc[df.sic=='', 'sic'] = pd.NA
        df.sic = df.sic.astype('Int32')
        df.loc[df.ein=='', 'ein'] = pd.NA
        df.ein = df.ein.astype('Int64')
        df.wksi = df.wksi.astype('bool')
        # df.wksi = df.wksi.astype('int32')
        df.period = pd.to_datetime(df.period, format='%Y%m%d', utc=True, errors='coerce')
        df.fy = pd.to_datetime(df.fy, format='%Y', utc=True, errors='coerce')
        df.filed = pd.to_datetime(df.filed, format='%Y%m%d', utc=True)
        df.accepted = pd.to_datetime(df.accepted, format='%Y-%m-%d %H:%M:%S', utc=True)
        df.prevrpt = df.prevrpt.astype('bool')
        df.detail = df.detail.astype('bool')
        # df.prevrpt = df.prevrpt.astype('int32')
        # df.detail = df.detail.astype('int32')
        df.nciks = df.nciks.astype('int32')
        
        cols = df.columns.tolist()
        cols = cols[0:3] + [cols[-1]] + cols[3:-1]
        df = df[cols]
    elif tbl=='num':
        # documentation wrongly lists coreg as NUMERIC length 256.  It is ALPHANUMERIC.
        df.ddate = pd.to_datetime(df.ddate, format='%Y%m%d', utc=True)
        df.qtrs = df.qtrs.astype('int32')
        df.loc[df.value=='', 'value'] = pd.NA
        df.value = df.value.astype('Float64')
    elif tbl=='tag':
        df.custom = df.custom.astype('bool')
        df.abstract = df.abstract.astype('bool')
    print(df.dtypes)
    display(df.head())

    buf = io.BytesIO()
    df.to_parquet(path=buf)
    buf.seek(0)
    bucket.upload_fileobj(Fileobj=buf,
                          Key=f'trino/{ingest_schema}/{tbl}/{qtr}.parquet')

    # Once we have all our parquet files in place, load up the table
    if qtr==quarters[-1]:
        table_check = engine.execute(f'drop table if exists {ingest_catalog}.{ingest_schema}.{tbl}')
        for row in table_check.fetchall():
            print(row)

        columnschema = create_table_schema_pairs(df)
        tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{tbl} (
{columnschema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket.name}/trino/{ingest_schema}/{tbl}/'
)
"""
        print(tabledef)

        table_create = engine.execute(tabledef)
        for row in table_create.fetchall():
            print(row)

        dataset_query = (f'SELECT * FROM {ingest_catalog}.{ingest_schema}.{tbl} limit 10')
        print(dataset_query)
        dataset = engine.execute(dataset_query)
        for row in dataset.fetchall():
            print(row)

In [ ]:
from datetime import datetime

for qtr in quarters:
    for ingest_table in [ 'sub', 'num', 'tag' ]:
        print(f'Ingesting table {ingest_table}; quarter = {qtr}; timestamp = {str(datetime.now())}')
        
        ingest_dera_table(qtr, ingest_table)

In [ ]:
# Borrow metadata code from DERA-iceberg if/when we need it

In [ ]:
tablenames = ['sub', 'num', 'tag', 'ticker']
l = []
for tbl in tablenames:
    qres = engine.execute(f'select count (*) from {ingest_catalog}.{ingest_schema}.{tbl}')
    l.append(qres.fetchall()[0])
print(list(zip(tablenames, l)))